In [5]:
import pandas as pd
import numpy as np

import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter

## Plot Word EDA

In [8]:
#import the extract function
import z_functions

In [2]:
#loading in spacy pipeline
nlp = spacy.load("en_core_web_lg")

In [18]:
#reading in data
movies = pd.read_csv('../data/genre_groups/x_final_df.csv')
movies.head(2)

,Unnamed: 0,imdb_id,title,overviews,join_director,join_cast,vote_average,vote_count,genre,user_plots,...,Mystery,Thriller,Western,Family,Sci-Fi,Romance,Film-Noir,Musical,Music,user_plot_keywords
0,0,tt0111161,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,FrankDarabont,TimRobbins MorganFreeman BobGunton,8.7,18845,Drama,['Two imprisoned men bond over a number of yea...,...,0,0,0,0,0,0,0,0,0,prison life murder wife imprisoned number year...
1,1,tt0068646,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",FrancisFordCoppola,AlPacino MarlonBrando JamesCaan,8.7,14225,Crime Drama,"[""An organized crime dynasty's aging patriarch...",...,0,0,0,0,0,0,0,0,0,family son head wedding business war life want...


In [37]:
#first 10 words in each user_plot_keywords as a list
user_plot_keywords = [words.split(' ')[:10] for words in movies['user_plot_keywords']]
user_plot_keywords[0]

['prison',
 'life',
 'murder',
 'wife',
 'imprisoned',
 'number',
 'years',
 'banker',
 'way',
 'befriends']

In [42]:
#creating dictionary of plot keywords, where the value is the frequency of each word
keyword_dict = {}

for group in user_plot_keywords:
    for word in group:
            if word not in keyword_dict:
                keyword_dict[word] = 1
            else:
                keyword_dict[word] += 1

In [61]:
word_freq = pd.DataFrame(list(keyword_dict.items()), columns=['words', 'count'])
word_freq.sort_values(by='count', ascending=False).head(30)

,words,count
1,life,1125
36,man,738
10,family,644
100,find,500
312,love,489
258,father,477
74,world,460
3,wife,450
564,woman,440
6,years,430


## ADJ EDA

In [63]:
def extract_adj(doc):
    key_words = []
    stopwords = list(STOP_WORDS)
    pos_tag = ['ADJ']

    for token in doc:
        if token.text in stopwords or token.text in punctuation:
            continue
        if token.pos_ in pos_tag:
            key_words.append(token.text)
            
    freq_word = Counter(key_words)
    freq_tups = freq_word.most_common(n=10)
    
    new_keywords = [word[0] for word in freq_tups]
    
    freq_string = ''
    for word in new_keywords:
        freq_string += word + ' ' 
    
    return freq_string

In [64]:
plots = [plot for plot in movies['user_plots']]
adjs = []

for plot in plots:
    adjs.append(extract_adj(nlp(plot)))

In [67]:
adjs[2]

'new criminal fine greatest sadistic rogue shady psychological physical mysterious '

In [74]:
plots[0]

'[\'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.\', "Chronicles the experiences of a formerly successful banker as a prisoner in the gloomy jailhouse of Shawshank after being found guilty of a crime he did not commit. The film portrays the man\'s unique way of dealing with his new, torturous life; along the way he befriends a number of fellow prisoners, most notably a wise long-term inmate named Red.", \'After the murder of his wife, hotshot banker Andrew Dufresne is sent to Shawshank Prison, where the usual unpleasantness occurs. Over the years, he retains hope and eventually gains the respect of his fellow inmates, especially longtime convict "Red" Redding, a black marketeer, and becomes influential within the prison. Eventually, Andrew achieves his ends on his own terms.\', "Andy Dufresne is sent to Shawshank Prison for the murder of his wife and her secret lover. He is very isolated and lonely at first, but r

In [72]:
t = nlp(plots[0])